In [1]:
import json
import yaml
from biofunctions.biofunctions import *
from datasets import Dataset
from transformers import T5Config, T5ForConditionalGeneration, TrainingArguments, Trainer


2022-11-23 16:30:41.028494: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-23 16:30:42.168931: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-23 16:30:42.169088: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-11-23 16:30:42.169098: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia 

In [2]:
with open('config.yaml','r') as f:
    config = yaml.safe_load(f)

In [3]:
with open('word_vocab.txt','r') as f:
    vocab = [line.strip() for line in f]

In [4]:
with open('input_output.json','r') as f:
    data = json.load(f)

In [5]:
len(data)

5980

In [6]:
data[5979]

{'input_text': '<CDR3>VATYYCLQSDSLPYTFGQGTK<ag>LPVLDRATCNRRTHHDGAITQRMMCAESNRRDSCKGDSGGPLVCGGVLEGVVTSGSRVCGNRKKPGIYTRV',
 'output_text': '<out>......................................................................._......................................................................._......................................................................._......................................................................._......................................................................._......................................................................._..........|||.........................................................._..........||||.|..............................................|........_.......||||||||||............................................|||......._.......|||||||||............|...............................|||||......_......||||||||||..........|||...............................|||||......_......||||||||.|.............................................|||.

In [7]:
my_dataset = Dataset.from_list(data)

In [8]:
my_dataset[5979]

{'input_text': '<CDR3>VATYYCLQSDSLPYTFGQGTK<ag>LPVLDRATCNRRTHHDGAITQRMMCAESNRRDSCKGDSGGPLVCGGVLEGVVTSGSRVCGNRKKPGIYTRV',
 'output_text': '<out>......................................................................._......................................................................._......................................................................._......................................................................._......................................................................._......................................................................._..........|||.........................................................._..........||||.|..............................................|........_.......||||||||||............................................|||......._.......|||||||||............|...............................|||||......_......||||||||||..........|||...............................|||||......_......||||||||.|.............................................|||.

In [9]:
my_dataset = my_dataset.train_test_split(test_size=0.2, shuffle=True, seed=1)

In [10]:
my_dataset

DatasetDict({
    train: Dataset({
        features: ['input_text', 'output_text', 'input_ids', 'labels'],
        num_rows: 4784
    })
    test: Dataset({
        features: ['input_text', 'output_text', 'input_ids', 'labels'],
        num_rows: 1196
    })
})

In [11]:
test_val = my_dataset['test'].train_test_split(test_size=0.5, shuffle=True, seed=1)
test_val

DatasetDict({
    train: Dataset({
        features: ['input_text', 'output_text', 'input_ids', 'labels'],
        num_rows: 598
    })
    test: Dataset({
        features: ['input_text', 'output_text', 'input_ids', 'labels'],
        num_rows: 598
    })
})

In [12]:
my_dataset['val'] = test_val['train']
my_dataset['test'] = test_val['test']
my_dataset

DatasetDict({
    train: Dataset({
        features: ['input_text', 'output_text', 'input_ids', 'labels'],
        num_rows: 4784
    })
    test: Dataset({
        features: ['input_text', 'output_text', 'input_ids', 'labels'],
        num_rows: 598
    })
    val: Dataset({
        features: ['input_text', 'output_text', 'input_ids', 'labels'],
        num_rows: 598
    })
})

In [13]:
data_collator =  DataCollator(vocab)

In [14]:
cfg = T5Config(**config['T5Config'])
model=T5ForConditionalGeneration(cfg)

In [15]:
TrainerArgs = TrainingArguments(
    output_dir='./results'
)

In [16]:
trainer = Trainer(
    model=model,
    args=TrainerArgs,
    data_collator=data_collator,
    train_dataset=my_dataset["train"],
    eval_dataset=my_dataset["test"]
)

In [17]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: input_text, output_text. If input_text, output_text are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/home/jahir/miniconda3/envs/paratope_venv/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4784
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1794
  Number of trainable parameters = 44077568


RuntimeError: [enforce fail at alloc_cpu.cpp:75] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 6325975296 bytes. Error code 12 (Cannot allocate memory)